# *학습 관련 기술들*

## 5. 적절한 하이퍼파라미터 값 찾기


- 신경망에서의 하이퍼파라미터는 각 층의 뉴런 수부터 배치크기, 매개변수 갱신시의 학습률과 가중치 감소 등 다양하게 등장
- 이 하이퍼파라미터 값에 따라 모델의 성능이 크게 좌우되기도 함
- 하이퍼파라미터 값 결정까지 많은 시행착오를 겪게 되는데, 아래 방식을 따라가면 효율적으로 탐색이 가능
___

### 1) 검증 데이터

- 그간 데이터를 training과 test set으로만 구분 하였지만, 하이퍼파라미터 조정을 위해서는 validation set 구비가 필요
> - training set:    매개변수 학습
> - validation set:  하이퍼파라미터 성능 평가
> - test set:        신경망의 범용 성능 평가

- training set의 약 20%정도로, 아래와 같이 분리

In [5]:
# coding: utf-8
import os
import sys

sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from data.mnist import load_mnist
from data.util import shuffle_dataset

In [8]:
(x_train, t_train), (x_test, t_test) = load_mnist()

x_train, t_train = shuffle_dataset(x_train, t_train)

validation_rate = 0.2
validation_num = int(x_train.shape[0] * validation_rate)

x_val = x_train[:validation_num]
t_val = t_train[:validation_num]
x_train = x_train[validation_num:]
t_train = t_train[validation_num:]

print(validation_num, x_val.shape, x_train.shape)

12000 (12000, 784) (48000, 784)


___
### 2) 하이퍼파라미터 최적화

- 하이퍼파라미터 최적화의 핵심은, 하이퍼파라미터의 최적 값이 존재하는 '범위'를 조금씩 줄여가는 것
- 신경망에서는 규칙적인 grid search 보다는 무작위로 샘플링하여 탐색하는 편이 좋은 결과에 빠르게 근사
- 하이퍼파라미터의 범위는 '대략적으로' 지정하는 것이 효과적\
- 예를 들어 0.001($10^{-3}$)에서 1,000($10^3$) 사이 같이 10의 거듭제곱 단위로 범위를 지정하며 이를 'log scale로 지정'한다고 함
- 추가적으로 하이퍼파라미터 최적화 때는 며칠 혹은 몇주까지 걸릴 수 있으므로 나쁠듯한 값은 일찍 포기하는 것이 효과적이며 에폭 값은 작게 설정
- 정리하자면 아래의 단계를 반복

> - **0 단계**

>   하이퍼파라미터 값의 범위 설정

> - **1 단계**

>   설정된 범위에서 하이퍼파라미터의 값을 무작위로 추출

> - **3 단계**

>   1단계에서 샘플링한 하이퍼파라미터 값을 사용하여 학습하고, 검증 데이터로 정확도를 평가 (에폭은 작게!)

> - **4 단계**

>   1단계와 2단계를 특정 횟수 (100회 등) 반복하며, 정확도의 결과를 보며 하이퍼 파라미터의 범위를 좁힘

- 위의 방식을 실용성에 기반하였으나, 좀 더 세련된 기법을 원한다면 **Bayesian optimization(베이즈 최적화)** 사용 권장
___

### 3) 하이퍼파라미터 최적화 구현하기

- 하이퍼파라미터의 검증은 그 값을 ($10^{-3} ~ 10^3$) 같은 로그스케일 범위에서 무작위로 추출

In [9]:
weight_decay = 10 ** np.random.uniform(-8, -4)
lr = 10 ** np.random.uniform(-6, -2)


2.985101898538857e-06


![](image/uniform_pdf.gif)

- np.random.uniform은 특정 구간에서 동일한 확률을 갖음
- 위와 같이 무작위로 추출한 값을 사용하여 학습을 수행
- 실험 결과는 아래와 같음
![](image/fig 6-24.png)

- 위처럼 학습 추이를 정확도가 높은 순서로 나열
- best 1부터 best 5까지의 학습 결과는 아래와 같음

> Best-1 (val acc:0.83) | lr:0.00920, weight decay: 3.86e-07

> Best-2 (val acc:0.78) | lr:0.00956, weight decay: 6.04e-07

> Best-3 (val acc:0.77) | lr:0.00571, weight decay: 1.27e-06

> Best-4 (val acc:0.74) | lr:0.00626, weight decay: 1.43e-05

> Best-5 (val acc:0.73) | lr:0.00520, weight decay: 8.97e-06

- 결과를 해석하자면 학습이 잘 진행될 때의 학습률은 0.001~0.01, 가중치 감소 계수는 $10^{-8}~10^{-6}$정도로 추정 가능
- 위 범위 내에서 다시 샘플링 후 동일한 작업 반복